# pull national flu ILI (Influenza like Ilness)

In [28]:
import requests
import pandas as pd

VALID_ENDPOINTS = {"fluview", "fluview_clinical", "flusurv", "fluview_meta"}
ENDPOINT_MAIN_PARAM = {
    "fluview": "regions",
    "fluview_clinical": "regions",
    "flusurv": "locations",
    "fluview_meta": None
}

def fetch_epidata(endpoint, start_week, end_week, **kwargs):
    """
    Fetch epidata using Delphi Epidata API. Returns a pandas DataFrame.

    Parameters:
    - endpoint: str, one of "fluview", "fluview_clinical", "flusurv", "fluview_meta"
    - start_week: str, in format "YYYYWW" (e.g. "202001" for the first week of 2020)
    - end_week: str, in format "YYYYWW" (e.g. "202052" for the last week of 2020)
    - Main parameter (required for all endpoints except fluview_meta):
        fluview / fluview_clinical → regions="nat"
        flusurv → locations="nat"
        - Additional parameters can be passed as keyword arguments (e.g. `issues`, `lag`, `auth`)

    """
    
    if endpoint not in VALID_ENDPOINTS:
        raise ValueError(f"Invalid endpoint: {endpoint}. Valid options are: {VALID_ENDPOINTS}")
        
    url = f"https://api.delphi.cmu.edu/epidata/{endpoint}/"
    params = {}

    # epiweek validation
    if endpoint != "fluview_meta" and (start_week is None or end_week is None):
        raise ValueError("start_week and end_week are required for this endpoint.")
    
    params["epiweeks"] = f"{start_week}-{end_week}"
    
    # validate main parameter
    main_param = ENDPOINT_MAIN_PARAM[endpoint]
    if main_param is not None:
        if main_param not in kwargs.keys():
            raise ValueError(f"Missing required parameter: {main_param} for endpoint {endpoint}.")
        params[main_param] = kwargs[main_param]

    # add addional parameters if provided    
    params.update(kwargs)
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json().get("epidata", [])
        return pd.DataFrame(data)
    else:
        print(f"Error fetching data: {response.status_code}. Response: {response.text}")
        return pd.DataFrame()
    

# example usage
if __name__ == "__main__":
    df = fetch_epidata(
        endpoint="fluview",
        start_week="202001",
        end_week="202052",
        regions="nat"
    )
    display(df.head())



,release_date,region,issue,epiweek,lag,num_ili,num_patients,num_providers,num_age_0,num_age_1,num_age_2,num_age_3,num_age_4,num_age_5,wili,ili
0,2021-10-08,nat,202139,202001,91,88731,1426691,2970,21594,23392,None,27655,9209,6881,5.90066,6.21936
1,2021-10-08,nat,202139,202002,90,75614,1492251,3002,15564,22756,None,23634,8196,5464,4.94020,5.06711
2,2021-10-08,nat,202139,202003,89,79783,1489132,2995,16587,29668,None,21413,7386,4729,5.33135,5.35768
3,2021-10-08,nat,202139,202004,88,90150,1496826,2997,18574,35275,None,23658,7973,4670,6.07822,6.02274
4,2021-10-08,nat,202139,202005,87,108307,1575900,2995,20299,44117,None,28889,9527,5475,6.74724,6.87271


In [1]:
import requests
import pandas as pd

# Base API URL
url = "https://api.delphi.cmu.edu/epidata/fluview/"

# Query parameters
params = {
    "regions": "nat",           # national level
    "epiweeks": "201701-202352", # range of epiweeks
    "format": "json"
}

# Send GET request
response = requests.get(url, params=params)

# Check HTTP status
if response.status_code == 200: # OK
    data = response.json()
    # Extract the actual data
    if "epidata" in data:
        df = pd.DataFrame(data["epidata"])
        print(df.head())
    else:
        print("No data found:", data)
else:
    print("Error:", response.status_code)

No data found: [{'release_date': '2018-10-05', 'region': 'nat', 'issue': 201839, 'epiweek': 201701, 'lag': 90, 'num_ili': 24178, 'num_patients': 785080, 'num_providers': 2028, 'num_age_0': 6716, 'num_age_1': 6786, 'num_age_2': None, 'num_age_3': 5673, 'num_age_4': 2773, 'num_age_5': 2230, 'wili': 3.09232, 'ili': 3.07969}, {'release_date': '2018-10-05', 'region': 'nat', 'issue': 201839, 'epiweek': 201702, 'lag': 89, 'num_ili': 24904, 'num_patients': 824374, 'num_providers': 2001, 'num_age_0': 6022, 'num_age_1': 8847, 'num_age_2': None, 'num_age_3': 5315, 'num_age_4': 2639, 'num_age_5': 2081, 'wili': 3.07623, 'ili': 3.02096}, {'release_date': '2018-10-05', 'region': 'nat', 'issue': 201839, 'epiweek': 201703, 'lag': 88, 'num_ili': 28759, 'num_patients': 845666, 'num_providers': 2033, 'num_age_0': 6403, 'num_age_1': 12121, 'num_age_2': None, 'num_age_3': 5588, 'num_age_4': 2717, 'num_age_5': 1930, 'wili': 3.50708, 'ili': 3.40075}, {'release_date': '2018-10-05', 'region': 'nat', 'issue': 20

In [4]:
# using their package
# Import
from epidatpy import CovidcastEpidata, EpiDataContext, EpiRange
# Fetch data
epidata = EpiDataContext()
res = epidata.pub_fluview(regions="nat", epiweeks=EpiRange(201501, 201510))
print(res)

EpiDataCall(endpoint=fluview/, params={'regions': 'nat', 'epiweeks': '201501-201510'})


In [7]:
# manually fetch data
# fx params
regions = ["nat", "MA", "MN", "CA"]
regions_joined = ",".join(regions)
start_epiweek = 201501
end_epiweek = 202352

# statatic params
url = "https://api.delphi.cmu.edu/epidata/fluview/"
params = {
    "regions": regions_joined,
    "epiweeks": f"{start_epiweek}-{end_epiweek}",
    "format": "json"
}
response = requests.get(url, params=params)

response.status_code
response.json()

[{'release_date': '2017-10-24',
  'region': 'ca',
  'issue': 201740,
  'epiweek': 201501,
  'lag': 143,
  'num_ili': 1865,
  'num_patients': 46662,
  'num_providers': 157,
  'num_age_0': None,
  'num_age_1': None,
  'num_age_2': None,
  'num_age_3': None,
  'num_age_4': None,
  'num_age_5': None,
  'wili': 3.99683,
  'ili': 3.99683},
 {'release_date': '2017-10-24',
  'region': 'ma',
  'issue': 201740,
  'epiweek': 201501,
  'lag': 143,
  'num_ili': 468,
  'num_patients': 26555,
  'num_providers': 47,
  'num_age_0': None,
  'num_age_1': None,
  'num_age_2': None,
  'num_age_3': None,
  'num_age_4': None,
  'num_age_5': None,
  'wili': 1.76238,
  'ili': 1.76238},
 {'release_date': '2017-10-24',
  'region': 'mn',
  'issue': 201740,
  'epiweek': 201501,
  'lag': 143,
  'num_ili': 163,
  'num_patients': 4474,
  'num_providers': 15,
  'num_age_0': None,
  'num_age_1': None,
  'num_age_2': None,
  'num_age_3': None,
  'num_age_4': None,
  'num_age_5': None,
  'wili': 3.64327,
  'ili': 3.64327

In [6]:
data = pd.DataFrame(response.json())
data.head()

,release_date,region,issue,epiweek,lag,num_ili,num_patients,num_providers,num_age_0,num_age_1,num_age_2,num_age_3,num_age_4,num_age_5,wili,ili
0,2017-10-24,ca,201740,201501,143,1865,46662,157,NaN,NaN,None,NaN,NaN,NaN,3.99683,3.99683
1,2017-10-24,ma,201740,201501,143,468,26555,47,NaN,NaN,None,NaN,NaN,NaN,1.76238,1.76238
2,2017-10-24,mn,201740,201501,143,163,4474,15,NaN,NaN,None,NaN,NaN,NaN,3.64327,3.64327
3,2017-10-24,nat,201740,201501,143,31483,771835,1958,7160.0,9589.0,None,8072.0,3614.0,3048.0,4.21374,4.07898
4,2017-10-24,ca,201740,201502,142,2065,47675,152,NaN,NaN,None,NaN,NaN,NaN,4.33141,4.33141
